## Premise: 

A babble tape is a digital file meant to be played in the background during
conversations. The file is complex. Forty voice tracks run simultaneously
(thirty-two in English, eight in other languages), and each track is compressed
in frequency and time to produce additional “voices” that fill the entire fre-
quency spectrum. There are also various non-human mechanical noises, and
a periodic supersonic burst (inaudible to adult listeners) engineered specifi-
cally to interfere with the automatic gain-control system of an eavesdropping
device configures itself to best pick up an audio signal. Most pertinent for
present purposes, the voices on a babble tape used by an attorney include
those of the client and the attorney themselves. The dense mélange of voices
increases the difficulty of discerning any single voice.


## Step 1: 

If you would like to make your onw babble tape, please record a minute of you talking with no background noise. If you prefer to read a default text, you may read the phonetic pangram text below: 

A pangram is a sentence or phrase that contains all the letters of the alphabet, at least once. Now, phonetic pangrams are sentences that contain all forty sounds of English i.e. they use all the phonemes, or phones, of English (rather than alphabetic characters) (sourced from quora: https://www.quora.com/Is-there-a-text-that-covers-the-entire-English-phonetic-range) .

Here is the text :

    "That quick beige fox jumped in the air over each thin dog. Look out, I shout, for he's foiled you again, creating chaos."

    "Are those shy Eurasian footwear, cowboy chaps, or jolly earthmoving headgear?"

    "The hungry purple dinosaur ate the kind, zingy fox, the jabbering crab, and the mad whale and started vending and quacking."

    "With tenure, Suzie’d have all the more leisure for yachting, but her publications are no good."

    "Shaw, those twelve beige hooks are joined if I patch a young, gooey mouth."

    "The beige hue on the waters of the loch impressed all, including the French queen, before she heard that symphony again, just as young Arthur wanted." 
    

## Step 2:

Convert your audio file into a wav file
Use dynamic range compression to fix very loud and very quiet parts of the audio recording (info from more info from: https://medium.com/@jud.dagnall/dynamic-range-compression-for-audio-with-ffmpeg-and-compand-621fe2b1a892)
The ffmpeg volume mapping was a follows:

    - 80/-900: Remove the really quiet stuff.
    -45/-15: Make the quietest part of the audience questions pretty clear(a 3x increase). You will likely need to fiddle with this if there’s a lot of audience noise, chairs moving, etc… that you don’t want to hear.
    -27/-9: make the medium part of the questions easy to hear.
    -5/-5: Keep the normal to loud voice unchanged (for now)
    20/20: Just an extra anchor point to keep the loud stuff loud (for now)


In [ ]:
!ffmpeg -i recording.m4a  -filter_complex "compand=attacks=0:points=-80/-900|-45/-15|-27/-9|-5/-5|20/20" recording.wav

## Step 3:

Slide up audio based on onset segements (using librosa's onset detection algorithm)

In [ ]:
import pandas as pd
import numpy as np
import librosa
import random
from pydub import AudioSegment
from pysndfx import AudioEffectsChain


def onset_detection_timestamps(soundFile, sr=44100):
    """Takes the path to an audio file
    and returns the list of start and stop times for that audio file
    as a frame rate

    Args:
        fileName (string): The path to an audio file
        sr (int, optional): The sample rate of the audio file. Defaults to 44100.

    Returns:
        [(int, int)]: A list of start and stop times for each sound change
    """
    y, sr = librosa.load(soundFile, sr=sr)
    C = np.abs(librosa.cqt(y=y, sr=sr))
    o_env = librosa.onset.onset_strength(sr=sr, S=librosa.amplitude_to_db(C, ref=np.max))
    onset_time = librosa.onset.onset_detect(onset_envelope=o_env, sr=sr, units='time', backtrack=True)
    return np.round(onset_time, 2).tolist()

In [2]:
# define file names:
input_audio_file = 'recording.wav'
output_babble_audio_file = "babble_me.wav"


#get the onset detection starting time stamps
times = onset_detection_timestamps(input_audio_file, sr=44100)

#break up audio into onset segments
audio = AudioSegment.from_wav(input_audio_file)

start = 0
audio_chunks = []
count = 0

for idx,t in enumerate(times):
    '''
    - break up audio into onset segments
    - if the audio segment is less than 0.3 seconds, then stitch it together with the next segment
    
    '''
    end = times[idx]*1000 #pydub works in millisec
    duration = (end - start)/1000
    
    if duration>0.3:
        audio_chunk=audio[start:end]
        audio_chunk.export("split_audio/Phonetic pangrams all_{}.wav".format(count), format="wav")
        audio_chunks.append(audio_chunk)
        start = end
        count += 1
    else:
        pass
        

## Step 4

- Combine the segemented audio (in a random order) into a single audio file
- Create multiple versions of the randomized combined segemented audio
- Overlay the different versions and apply panning to the different layers

In [3]:
#now stitch together the segments in a random order and create x versions of the audio files so that we can eventually layer them together
combined_sounds_list = []
layers = 7
for i in range(0, layers):
    '''
    - Stitch together the segments in a random order
    - Create x versions of the audio files so that we can eventually layer them together

    '''
    random.shuffle(audio_chunks)
    for idx,t in enumerate(audio_chunks):
        if idx == 0:
            combined_sounds = audio_chunks[idx]
        else:  
            combined_sounds += audio_chunks[idx]
    combined_sounds_list.append(combined_sounds)
    
    
for i in range(0,layers-1):
    '''
    Overlaying the different versions of the audio together
    '''
    if i == 0:
        _overlay = combined_sounds_list[i].low_pass_filter(3000).overlay(combined_sounds_list[i+1])#
    elif i%2:
        _overlay = _overlay.pan(-0.5).overlay(combined_sounds_list[i+1].low_pass_filter(3000).pan(0.5))
    else:
        _overlay = _overlay.overlay(combined_sounds_list[i+1].low_pass_filter(3000))


#That's it! Now writing the audio file

file_handle = _overlay.export(output_babble_audio_file, format="wav")
audio = AudioSegment.from_wav(output_babble_audio_file)
audio

## Step 5 (optional)

Add a bit of reverb to the recording

In [11]:
#Just adding a bit of reverb

infile = output_babble_audio_file
outfile = output_babble_audio_file[:-4]+"_reverb.wav"

fx = (
    AudioEffectsChain()
    .reverb()
)

# Apply phaser and reverb directly to an audio file.
fx(infile, outfile)

# Or, apply the effects directly to a ndarray.
y, sr = librosa.load(infile, sr=44100)
y = fx(y)

# Apply the effects and return the results as a ndarray.
y = fx(infile)

# Apply the effects to a ndarray but store the resulting audio to disk.
fx(y, outfile)

audio = AudioSegment.from_wav(outfile)
audio